# Dependencias

In [26]:
import mlflow
import os
import shutil
import boto3
from datetime import datetime


S3_CLIENT = boto3.resource('s3')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
MLFLOW_CLIENT = mlflow.tracking.MlflowClient()

REGISTERED_MODELS = ["Hands"]
MODELS = {}


def downlod_model(bucket_name, remoteDirectory_name):

    bucket = S3_CLIENT.Bucket(bucket_name)

    for obj in bucket.objects.filter(Prefix=remoteDirectory_name):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)


def update_models(version=-1, remove_old_versions=False):

    update = {}

    for model_name in REGISTERED_MODELS:
        model = None
        update[model_name] = 0
        for mv in MLFLOW_CLIENT.search_model_versions(f"name='{model_name}'"):
            mv_bckp = mv
            mv = dict(mv)
            if version == mv['version'] or (version == -1 and mv['current_stage'] == 'Production'):
                mv['last_updated_timestamp'] = str(datetime.fromtimestamp(int(mv['last_updated_timestamp'] / 1000)))
                bucket = mv['source'].split('//')[1].split('/')[0]
                folder = mv['source'].split('//')[1].split('/')[1]
                if os.path.exists(os.path.join('./models', folder)):
                    print("Load existing model...")
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                else:
                    print("Downloading model...")
                    downlod_model(bucket, folder)
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                    if remove_old_versions and os.path.exists('./models'):
                        shutil.rmtree('./models')
                    if not os.path.exists('./models'):
                        os.mkdir('./models')
                    shutil.move(os.path.join(os.getcwd(), folder), './models')
                    update[model_name] = 1
                print("Using model {name} v{version} ({current_stage}) updated at {last_updated_timestamp}".format(**mv))
                #response = {k: v for k, v in mv.items() if v}
                break
        if model:
            MODELS[model_name] = (model, mv_bckp)

    return update


def get_model(model_name):

    return MODELS.get(model_name, None)

# Descargar la última versión del modelo

In [28]:
os.chdir('/home/ubuntu/tfm/standalone')
versions = [10, 11]
for version in versions:
    update_flag = update_models(version)
    model_path, model_meta = get_model('Hands')
    print(MLFLOW_CLIENT.get_run(model_data.run_id))

Load existing model...
Using model Hands v10 (Staging) updated at 2020-12-29 21:58:48
<Run: data=<RunData: metrics={'loss': 22.5849361419678,
 'lr': 0.0001,
 'stopped_epoch': 64.0,
 'val_loss': 22.2823333740234}, params={'augmentation': 'True',
 'baseline': 'None',
 'batch_size': '16',
 'early_stop_patience': '5',
 'epochs': '100',
 'initial_epoch': '0',
 'learning_rate': '0.0001',
 'min_delta': '0',
 'monitor': 'val_loss',
 'normalize': 'True',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_decay': '0.0',
 'opt_epsilon': '1e-07',
 'opt_learning_rate': '0.0001',
 'opt_name': 'Adam',
 'patience': '3',
 'reduce_lr_factor': '0.1',
 'reduce_lr_patience': '3',
 'restore_best_weights': 'False',
 'standarize': 'True',
 'steps_per_epoch': '115',
 'validation_split': '0.2',
 'validation_steps': '28'}, tags={'dataset': 'vott-csv-export-augmented-all',
 'mlflow.log-model.history': '[{"run_id": "82f298920f8646ef89bd4313ef8202d9", '
                             '"artif

In [29]:
os.listdir('/home/ubuntu/tfm/standalone/models')

['e97e8a8ee4854e428e66715bef1f6dee', '82f298920f8646ef89bd4313ef8202d9']